### Kalman Smoothed Altimeter Analysis (Examples)

This notebook provides visualizations for synoptic and mesoscale pressure analyses generated using LatticeKrig and a Kalman smoother to ensure temporal consistency between independent analyses. Pre-generated MADIS pressure analyses, created with MADIS research observations, are used to produce movies that show the evolution of altimeter (sea level pressure) on 14 April, 2018 and on 14-15 of May, 2018. 

1. Import relevant Python libraries and setup cartopy/colortables

2. Retrieve MADIS (Kalman smoothed) pressure analyses and plot the synoptic distribution of pressure over a 24-h period.  

3. Plot and save Kalman smoothed pressure analysis over the Mid-Atlantic region during two high-impact weather events (back to back derechoes). Produce a movie depicting the evolution fo the pressure and reflectivity over the two-day period.

4. Demonstrate how mesoscale pressure perturbations are extracted from pressure analyses using band-pass filtering. Plot and save mesoscale pressure perturbations and composite reflectivity analyses over the two-day period from (3). From the saved images, produce a movies showing mesoscale pressure perturbations associated with convection.

In [89]:
### ---- (1) ---- ####
#Import Python libraries

import os
import xarray as xr
import matplotlib
import cmasher as cmr
from matplotlib import pyplot as plt
import numpy as np
import funcs
import sys
import colorcet as cc
import cmasher as cmr
from datetime import datetime,timedelta
from cartopy.feature import NaturalEarthFeature,BORDERS,LAKES,COLORS
import cartopy.crs as crs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from metpy.plots import colortables
from scipy import ndimage
from scipy import signal
from scipy.signal import butter, lfilter
import multiprocessing
from joblib import Parallel,delayed

#Retrieve perceptually uniform colorbar from colorcet
cmapp = cc.cm.rainbow_bgyrm_35_85_c71

#Set format for datetime objects
fmt = '%Y%m%d_%H%M'

# Download/add state and coastline features for cartopy 
states = NaturalEarthFeature(category="cultural", scale="10m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")

land_50m = NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='k',
                                        facecolor='none')

#Define function to add map data to matplotlib plot
def add_map(ax,clr,lw):
    ax.add_feature(states)
    ax.add_feature(BORDERS)
    ax.add_feature(land_50m)
    ax.add_feature(states,edgecolor=clr,lw=lw)
    ax.add_feature(LAKES, edgecolor=clr)

#Define function to add latitude/longitude grid lines to cartopy/matplotlib plot
def add_gridlines(ax,xl,yl,clr, fs):
    gl = ax.gridlines(crs=crs.PlateCarree(), draw_labels=True,
                      linewidth=0.25, color=clr, alpha=1, linestyle='--')

    gl.xlabels_bottom = xl
    gl.xlabels_top = False
    gl.ylabels_left = yl
    gl.ylabels_right = False

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': fs, 'color': clr}
    gl.ylabel_style = {'size': fs, 'color': clr}
    return gl

#Get Composite Reflectivity colormap from metpy
ctable1 = 'NWSStormClearReflectivity'
cmapp = cc.cm.rainbow_bgyrm_35_85_c71
norm, cmapp_radar = colortables.get_with_steps(ctable1, 244, 244)

#Increase with of notebook to fill screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#Define function to mask pressure analyses over water
def mask_grid(arr):
    arr = np.ma.masked_where(landsea==0,arr)
    return arr

#Define function to read and subset a land/sea boolean grid
def get_landsea():
    ds_land = xr.open_dataset('../data/Static/landsea.nc')
    ds_land = funcs.subset(ds_land,minLat,maxLat,minLng,maxLng)
    landsea = ds_land['LANDSEA'].values
    landsea = np.pad(landsea, ((0,1),(0,1)), 'edge')
    ds_land.close()
    return landsea

In [90]:
#---- (2) ---- #

#Set date, date format, and observation type
day = '20180414'
otyp = 'altimeter'
fmt = '%Y%m%d_%H%M'

#Define bounding box for analysis
minLng = -105.5; maxLng = -70.5; minLat = 28.5; maxLat = 48.5
#Get land/sea boolean within bounding box
landsea = get_landsea()

#Retrieve pre-generated Kalman smoothed LatticeKrig pressure analysis for 14 April, 2018.
ds_all = xr.open_dataset('../data/KF/kfmadis_full_altimeter_'+day+'.nc')
#Convert analysis time to list of datetime objects
dts = ds_all['Valid'].values 
dtlist = [datetime.utcfromtimestamp(d/1e9).strftime(fmt) for d in dts.tolist()]

#Retrieve altimeter grid and latitude/longitude dims
alts_kf = ds_all['altimeter_rts'].values
ygrid = ds_all['longitude'].values; xgrid = ds_all['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Generate 2D coordinate grid for contouring

In [91]:
#Set lower and upper bounds for pressure colorbar
mmin = round(np.nanmin(alts_kf)-0.5,0)
mmax = round(np.nanmax(alts_kf)+0.5,0)

#Define function to plot pressure anlyses and save to a png file in the Plots directory
def plot_one(d,ddate):    
    #Mask altimeter analysis over water
    alts_kf_2d = mask_grid(alts_kf[d])
    #Smooth altimeter analysis for contouring (use higher smoothing level for synoptic analysis)
    alts_kf_2d_smooth = ndimage.gaussian_filter(alts_kf[d],sigma=5)
    
    #Initiatlize figure
    fig =plt.figure(figsize=(18,10))
    ax1 = plt.subplot(111,projection=crs.PlateCarree()) #Set projection
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',20) #Add grid lines and x/y labels  
    im = ax1.imshow(alts_kf_2d,origin='lower',extent=[minLng,maxLng,minLat,maxLat],cmap=cmapp,vmin=mmin,vmax=mmax)
    #Contour pressure every 2 hPa
    CS = ax1.contour(X,Y,mask_grid(alts_kf_2d_smooth),levels=np.arange(mmin,mmax+2,2),colors='k',alpha=1)
    ax1.clabel(CS, CS.levels, inline=True, fmt="%1.f", fontsize=16, colors='k') #Add contour labels
    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('5-min MADIS Altimeter Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($hPa$)',y=1.02,fontsize=20) #Set colorbar title
    cb.ax.tick_params(labelsize=20) #Set colorbar tick size
    fig.canvas.draw()
    plt.tight_layout()
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
    plt.savefig('../data/Plots/'+day+'/kfalts_'+dd+'.png')
    plt.close()

#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_one)(d,ddate) for d,ddate in enumerate(dtlist))

#Alternatively, perform plotting sequentially on a single core
#for d,ddate in enumerate(dtlist):
#    print('Plotting: '+str(ddate))
#    plot_one(d,ddate)

In [92]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day+'/kfalts_'+day+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day+'/kfalts_'+day+'.mp4')
#Create mp4 movie from 5-min pressure anlayses saved as pngs
os.system('ffmpeg -r 9 -f image2 -s 1920x1080 -i ../data/Plots/'+day+'/kfalts_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day+'/kfalts_'+day+'.mp4')
#(Below) Display MADIS pressure analysis for 14 April, 2018

0

In [93]:
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src = "../data/Plots/20180414/kfalts_20180414.mp4" type="video/mp4">
</video></div>

In [102]:
#---- (3) ---- #

#Define dates of analysis and observation type
day1 = '20180514'
day2 = '20180515'
otyp = 'altimeter'

#Define bounding box
minLng = -83.0; maxLng = -70.5; minLat = 38.5; maxLat= 45.0
#Get land/sea boolean within bounding box
landsea = get_landsea()
landsea = landsea[:-1,:]

#Retrieve pressure analyses for each day 14-15 of May, 2018
ds1 = xr.open_dataset('../data/KF/kfmadis_full_altimeter_'+day1+'.nc')
ds2 = xr.open_dataset('../data/KF/kfmadis_full_altimeter_'+day2+'.nc')
#Combine pressure analysis from each day into a single xarray dataset
ds_all = xr.concat([ds1,ds2],'Valid')

#Retrieve latitude/longitude from dataset
ygrid = ds_all['longitude'].values; xgrid = ds_all['latitude'].values
X,Y = np.meshgrid(ygrid,xgrid) #Create 2d coordinates for contour plotting

#Retrieve composite reflectivity for each day: 14-15 of May, 2018
dsr1 = xr.open_dataset('../data/Radar/cref_'+day1+'.nc')
dsr2 = xr.open_dataset('../data/Radar/cref_'+day2+'.nc')
#Combine reflectivity analysis from each day into a single xarray dataset
dsr_all = xr.concat([dsr1,dsr2],'Valid')

#Convert observation times into list of datetime objects
dts = ds2['Valid'].values
dtlist = [datetime.utcfromtimestamp(d/1e9).strftime(fmt) for d in dts.tolist()]

#Get pressure and reflectivity analyses
alts_kf = ds2['altimeter_rts'].values
refl = dsr2['REFL'].values

In [86]:
#Set base font size
matplotlib.rcParams.update({'font.size': 20})

#Set lower and upper bounds for pressure colorbar
mmin = round(np.nanmin(alts_kf)-0.5,0)
mmax = round(np.nanmax(alts_kf)+0.5,0)

#Define function to plot/save pressure and composite reflectivity analysis 
def plot_two(d,ddate):  
    #Get 5-min reflectivity and altimeter analysis
    rfl_2d = refl[d]
    #Mask altimeter anlaysis over water
    alts_kf_2d = mask_grid(alts_kf[d])
    #Smooth altimeter analysis for contouring
    alts_kf_2d_smooth = ndimage.gaussian_filter(alts_kf[d],sigma=2.5)
    
    #Initialize figure
    fig =plt.figure(figsize=(28,8))
    
    #Plot pressure analysis
    ax1 = plt.subplot(121,projection=crs.PlateCarree()) #define projection
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',22) #Add grid lines and x/y labels  
    im = ax1.imshow(alts_kf_2d,origin='lower',extent=[minLng,maxLng,minLat,maxLat],cmap=cmr.pride,vmin=mmin,vmax=mmax)
    #Contour pressure every 1 hPa
    CS = ax1.contour(X,Y,mask_grid(alts_kf_2d_smooth),levels=np.arange(mmin,mmax+1,1),colors='k',alpha=1)
    ax1.clabel(CS, CS.levels, inline=True, fmt="%1.f", fontsize=16, colors='k') #Label contours
    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('MADIS Altimeter '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=26)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($hPa$)',y=1.02,fontsize=22) #Set colorbar title
    cb.ax.tick_params(labelsize=22) #Set colorbar tick size
    
    #Plot composite reflectivity analysis
    ax2 = plt.subplot(122,projection=crs.PlateCarree())
    add_map(ax2,'dimgray',1) #Add States/borders
    add_gridlines(ax2,True,False,'k',22) #Add grid lines and x/y labels  
    im = ax2.imshow(np.ma.masked_where(rfl_2d<=20,rfl_2d),origin='lower',extent=[minLng,maxLng,minLat,maxLat],cmap=cmapp_radar,vmin=-32,vmax=90,zorder=5,alpha=0.8)
    #Set grid bounds
    ax2.set_xlim([minLng,maxLng])
    ax2.set_ylim([minLat,maxLat])
    ax2.set_title('Composite Reflectivity',fontsize=26)
    cb=plt.colorbar(im,fraction=0.023)
    cb.ax.set_title('($hPa$)',y=1.02,fontsize=22)
    cb.ax.tick_params(labelsize=22)
        
    #Save image with %03d format for animation with ffmpeg
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
        
    plt.suptitle('5-min Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    fig.canvas.draw()
    plt.tight_layout()
    plt.savefig('../data/Plots/'+day2+'/kfcomb_'+dd+'.png')
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_two)(d,ddate) for d,ddate in enumerate(dtlist))

#Alternatively, perform plotting sequentially on a single core
#for d,ddate in enumerate(dtlist):
#    print('Plotting: '+str(ddate))
#    plot_two(287,dtlist[287]) #ddate) #ddate)
#    break

In [87]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day2+'/kfcomb_'+day2+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day2+'/kfcomb_'+day2+'.mp4')
#Create mp4 movie from 5-min pressure/reflectivity anlayses saved as pngs
os.system('ffmpeg -r 12 -f image2 -s 1920x1080 -i ../data/Plots/'+day2+'/kfcomb_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day2+'/kfcomb_'+day2+'.mp4')
#(Below): Display video

0

In [88]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../data/Plots/20180515/kfcomb_20180515.mp4" type="video/mp4">
</video></div>

In [104]:
#Get dimensions of pressure anlaysis
ys = alts_kf.shape[1] #lat dim
xs = alts_kf.shape[2] #lng dim

#Set upper and lower limits of band pass filter
highcut = 1/(3600*2)
lowcut = 1/(3600*6)
fs = 1/300.0 #Set temporal frequency of analysis (in hertz)
order=2

#Define highpass/bandpass filter function
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs #nyquist frequency
        low = lowcut / nyq #lower limit filter
        high = highcut / nyq #upper limit of filter
        b, a = butter(order, [low, high], btype='band') #Perform bandpass
        #b, a = butter(order, [high], btype = 'highpass')
        return b, a

#Function to perform band pass filtering
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = signal.filtfilt(b, a, data)
        return y

#Filter time-series
def filter_ts(vvar):
        meso = butter_bandpass_filter(vvar, lowcut, highcut, fs, order)
        return meso

#Function to perform bandpass filtering of pressure time series at grid point (y,x)
def perform_filter(vvar,x,y):
    if (y == xs-1):
            print(x)
    
    #Get bandpass filtered time-series
    meso = filter_ts(vvar)
    return meso

#Get list of latitude/longitude pairs for every grid point in the analysis domain
xy_pair = [];
for i in range(0,ys):
    for j in range(0,xs):
        xy_pair.append((i,j))


#Get altimeter and reflectivity analyses over the two-day period (14-15, May 2018) 
alts_kf = ds_all['altimeter_rts'].values
refl = dsr_all['REFL'].values
        
#Perform bandpass filtering in parallel
num_cores = multiprocessing.cpu_count()
vvar_meso = Parallel(n_jobs=num_cores)(delayed(perform_filter)(alts_kf[:,x,y],x,y) for x,y in xy_pair)

#Convert list of analysis times to list of datetime objects
dts = ds_all['Valid'].values
dtlist = [datetime.utcfromtimestamp(d/1e9).strftime(fmt) for d in dts.tolist()]

#Reshape the bandpass filtered pressure dataset so its dimensions are (Time, Latitude, Longitude)
vvar_meso = np.float32(vvar_meso)
nshp = (ys,xs,len(dtlist))
vvar_meso = np.reshape(vvar_meso,nshp).T
vvar_meso = np.swapaxes(vvar_meso,1,2)

#Write bandpass filtered pressure perturbations to NetCDF
#ds = xr.Dataset()
#ds['altimeter_meso'] = xr.DataArray(vvar_meso,coords={'Valid':dts,'latitude':ds_all['latitude'].values,'longitude':ds_all['longitude'].values},dims=('Valid','latitude','longitude'))
#ds.to_netcdf('../data/KF/kfmeso_bpass_'+day2+'.nc')

In [106]:
#Set base font size
matplotlib.rcParams.update({'font.size': 20})

#Get land/sea boolean within bounding box
landsea = get_landsea()
landsea = landsea[:-1,:]

#Get latitude/longitude and 2D coordinates of analysis domain
xgrid = ds_all['longitude'].values; ygrid = ds_all['latitude'].values
X,Y = np.meshgrid(xgrid,ygrid)

#Define function to mesoscale pressure perturbations and composite reflectivity analysis
def plot_two(d,ddate):  
    #Start at 1200 UTC 14, May
    d = d+144
    #Retrieve 5-min reflectivity and altimeter analysis
    rfl_2d = refl[d]
    #Mask altimeter analysis over water
    vvar_meso_2d = mask_grid(vvar_meso[d])
    #Smooth altimeter analysis for contouring
    vvar_meso_2d_smooth = ndimage.gaussian_filter(vvar_meso_2d,sigma=2.5)
    
    #Initialize Figure
    fig =plt.figure(figsize=(26,8))
    
    #Plot mesoscale (bandpass) pressure perturbation analysis
    ax1 = plt.subplot(121,projection=crs.PlateCarree())
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',18) #Add grid lines and x/y labels  
    im = ax1.imshow(vvar_meso_2d,origin='lower',extent=[minLng,maxLng,minLat,maxLat],cmap=cmr.fusion_r,vmin=-2,vmax=2)
    #Contour positive and negative perturbations at 0.75 and -0.75 hPa, respectively 
    CS = ax1.contour(X,Y,vvar_meso_2d,levels=[0.75],colors='k',alpha=1)
    CS = ax1.contour(X,Y,vvar_meso_2d,levels=[-0.75],ls='--',colors='k',alpha=1)
    ax1.clabel(CS, CS.levels, inline=True, fmt="%1.2f", fontsize=14, colors='k') #add contour labels
    #Set grid bounds
    ax1.set_xlim([minLng,maxLng])
    ax1.set_ylim([minLat,maxLat])
    ax1.set_title('Mesoscale (2-6 h) Band-pass Altimeter',fontsize=22)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($hPa$)',y=1.02,fontsize=18) #Set colorbar title
    cb.ax.tick_params(labelsize=18) #Set colorbar tick size   
    
    #Plot composite reflectivity analysis
    ax2 = plt.subplot(122,projection=crs.PlateCarree())
    add_map(ax1,'dimgray',1) #Add States/borders
    add_gridlines(ax1,True,True,'k',18) #Add grid lines and x/y labels  
    im = ax2.imshow(rfl_2d,origin='lower',extent=[minLng,maxLng,minLat,maxLat],cmap=cmapp_radar,vmin=-32,vmax=90,zorder=2,alpha=0.8)
    #Set grid bounds
    ax2.set_xlim([minLng,maxLng])
    ax2.set_ylim([minLat,maxLat])
    ax2.set_title('Composite Reflectivity',fontsize=22)
    cb=plt.colorbar(im,fraction=0.023) #Shrink colorbar to fit plot height
    cb.ax.set_title('($dBZ$)',y=1.02,fontsize=18) #Set colorbar title
    cb.ax.tick_params(labelsize=18) #Set colorbar tick size
        
    #Save image with %03d format for animation with ffmpeg
    d = d-144
    if (d < 10):
        dd = '00'+str(d)
    elif ((d >= 10) and (d < 100)):
        dd = '0'+str(d)
    else:
        dd = str(d)
        
    plt.suptitle('5-min Analysis '+ddate[9:13]+' UTC '+ddate[6:8]+'/'+ddate[4:6]+'/'+ddate[0:4],fontsize=24)
    fig.canvas.draw()
    plt.tight_layout()
    plt.savefig('../data/Plots/'+day2+'/kfpert_'+dd+'.png')
    plt.close()
    
#Perform plotting in parallel (one plot - per core)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(plot_two)(d,ddate) for d,ddate in enumerate(dtlist[144:]))

In [107]:
#If animation (mp4 movie) already exists, remove it so ffmpeg won't ask to overwrite
if os.path.isfile('../data/Plots/'+day2+'/kfpert_'+day2+'.mp4'):
    os.system('rm -rf ../data/Plots/'+day2+'/kfpert_'+day2+'.mp4')
#Create mp4 movie from 5-min pressure perturbation / reflectivity anlayses saved as pngs
os.system('ffmpeg -r 12 -f image2 -s 1920x1080 -i ../data/Plots/'+day2+'/kfpert_%03d.png -c:v libx264 -pix_fmt yuv420p ../data/Plots/'+day2+'/kfpert_'+day2+'.mp4')
#(Below) display video

0

In [108]:
%%HTML
<div align="middle">
<video width="100%" controls>
      <source src = "../data/Plots/20180515/kfpert_20180515.mp4" type="video/mp4">
</video></div>